In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import time

from lib.utils import model_results_from_npz
from lib.projection import gram_schmidt, project_confidences
from lib.redistribute import redistribute_confidences_of_class

# ===== Dataset Config =====
# If you want explicit class names, load from your CIFAR-100 mapping.

CIFAR100_CLASSES = [
    'apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 
    'bed', 'bee', 'beetle', 'bicycle', 'bottle', 'bowl', 
    'boy', 'bridge', 'bus', 'butterfly', 'camel', 'can', 
    'castle', 'caterpillar', 'cattle', 'chair', 'chimpanzee', 
    'clock', 'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 
    'cup', 'dinosaur', 'dolphin', 'elephant', 'flatfish', 'forest', 
    'fox', 'girl', 'hamster', 'house', 'kangaroo', 'keyboard', 'lamp', 
    'lawn_mower', 'leopard', 'lion', 'lizard', 'lobster', 'man', 
    'maple_tree', 'motorcycle', 'mountain', 'mouse', 'mushroom', 
    'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear', 
    'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine', 
    'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose', 'sea', 
    'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake', 
    'spider', 'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'table', 
    'tank', 'telephone', 'television', 'tiger', 'tractor', 'train', 'trout', 
    'tulip', 'turtle', 'wardrobe', 'whale', 'willow_tree', 'wolf', 'woman', 'worm'
    ]
NUM_CLASSES = 100

SEEDS = [42, 602, 311, 637, 800, 543, 969, 122, 336, 93]   # example seeds
EPOCHS = 50
REMOVED_CLASSES = [
    4, 7, 6, 11, 14, 15, 19, 24, 23, 25, 29, 30, 
    31, 36, 35, 39, 44, 41, 49, 48, 51, 53, 59, 57, 60, 61, 
    68, 67, 72, 71, 76, 77, 80, 84, 88, 85, 92, 94, 99, 97] # CIFAR-100

#SEEDS = [311, 637, 969]   # example seeds
#EPOCHS = 200
#REMOVED_CLASSES = [14, 23, 35, 49, 53, 61, 68, 72, 88, 97]

# ===== Paths =====
DIR_ORI = "../results/cifar100/"
DIR_OUT = "../results/redis_cifar100/"
TIMER_DIR = "../analytics/CIFAR100/timer/"
os.makedirs(DIR_OUT, exist_ok=True)

# ===== Timer Store =====
timer_records = []

# ===== Redistribute Loop =====
for seed in SEEDS:
    file = f"cifar100_resnet18_s{seed}_e{EPOCHS}.npz"
    filepath = os.path.join(DIR_ORI, file)
    if not os.path.exists(filepath):
        print(f"Skip missing: {filepath}")
        continue

    # Load original once per seed
    model_ori = model_results_from_npz(filepath=filepath, num_classes=NUM_CLASSES)

    # Precompute averages once
    avg_confidences = np.zeros((NUM_CLASSES, NUM_CLASSES), dtype=np.float32)
    for i in range(NUM_CLASSES):
        avg_confidences[i] = np.mean(model_ori.confidences[i], axis=0)
    avg_confidences_ortho = gram_schmidt(avg_confidences)

    for removed_class in tqdm(REMOVED_CLASSES, desc=f"Seed {seed}", leave=False):
        start_time = time.perf_counter()

        proj_confidences = project_confidences(
            model_ori.confidences,
            avg_confidences_ortho,
            removed_class
        )

        avg_proj_confidences = np.zeros((NUM_CLASSES, NUM_CLASSES), dtype=np.float32)
        for i in range(NUM_CLASSES):
            avg_proj_confidences[i] = np.mean(proj_confidences[i], axis=0)

        # Redistribute per class
        redis = {}
        for cls in range(NUM_CLASSES):
            redis[cls] = redistribute_confidences_of_clas(
                conf=model_ori.confidences[cls],
                proj_conf=proj_confidences[cls],
                avg_proj_conf=avg_proj_confidences[cls],
                removed_class=removed_class,
            )

        end_time = time.perf_counter()
        # Record timing
        duration_sec = end_time - start_time
        timer_records.append({
            "seed": seed,
            "removed_class": removed_class,
            "removed_name": CIFAR100_CLASSES[removed_class],
            "duration_sec": duration_sec
        })
        
        # Concatenate back
        all_confs = np.vstack([redis[cls] for cls in range(NUM_CLASSES)])
        all_targets = np.concatenate([
            np.full(redis[cls].shape[0], cls, dtype=int) for cls in range(NUM_CLASSES)
        ])
        all_preds = np.argmax(all_confs, axis=1)

        # Save
        out_path = os.path.join(DIR_OUT, f"cifar_resnet_s{seed}_e{EPOCHS}_rd{removed_class}.npz")
        np.savez_compressed(out_path, preds=all_preds, targets=all_targets, confs=all_confs)
        print(f"[Saved] {out_path}")

timer_df = pd.DataFrame(timer_records)
timer_csv_path = os.path.join(TIMER_DIR, "redistribution_timer.csv")
timer_df.to_csv(timer_csv_path, index=False)
print(f"\nAll timers saved to {timer_csv_path}")


Seed 42:   2%|▎         | 1/40 [00:00<00:12,  3.15it/s]

[Saved] ../results/redis_cifar100/cifar_resnet_s42_e50_rd4.npz


Seed 42:   5%|▌         | 2/40 [00:00<00:10,  3.52it/s]

[Saved] ../results/redis_cifar100/cifar_resnet_s42_e50_rd7.npz


Seed 42:   8%|▊         | 3/40 [00:00<00:10,  3.47it/s]

...

Seed 93:  95%|█████████▌| 38/40 [00:08<00:00,  4.61it/s]

[Saved] ../results/redis_cifar100/cifar_resnet_s93_e50_rd94.npz


Seed 93:  98%|█████████▊| 39/40 [00:09<00:00,  4.35it/s]

[Saved] ../results/redis_cifar100/cifar_resnet_s93_e50_rd99.npz


[Saved] ../results/redis_cifar100/cifar_resnet_s93_e50_rd97.npz

All timers saved to ../analytics/CIFAR100/timer/redistribution_timer.csv
